# Excercise 1

## 1A: Dimensions

- Create a dimension set with two dimensions:
  - time (letter t) with items 2010, 2020, 2030 and 2040
  - element (letter e) with items Fe and Cu
- Print the results.

In [ ]:
import flodym as fd

dims = fd.DimensionSet(dim_list=[
    fd.Dimension(name="time", letter="t", items=[2010, 2020, 2030, 2040], dtype=int),
    fd.Dimension(name="element", letter="e", items=["Fe", "Cu"], dtype=str),
])

print(dims)

## 1B: Arrays

You get two numpy arrays with values: 


In [ ]:
import numpy as np

production_values = np.array([
    [11, 0.11],
    [12, 0.12],
    [13, 0.13],
    [12, 0.12],
])

loss_rate_values = np.array([0.1, 0.09, 0.08, 0.07])


- 1B1. Create two flodym arrays `production` (dimensions t, e) and `loss_rate` (dimension t) from them.
- 1B2. Calculate a new array `losses` (per year and element) as `production` times `loss_rate`
- 1B3. Print the copper ("Cu") losses over time


In [ ]:
# 1B1
production = fd.FlodymArray(dims=dims, values=production_values)
loss_rate = fd.FlodymArray(dims=dims[("t",)], values=loss_rate_values)

# 1B2
losses = production * loss_rate

# 1B3
print(losses["Cu"])


### Bonus:
- 1B4. Create and empty flodym array `total_losses` (per year, summed over elements)
- 1B5. Calculate `total_losses` as `production` times `loss_rate`.
- 1B6. Calculate the difference between `losses` and `total_losses` and print the result. What has happened?


In [ ]:
# 1B4
total_losses = fd.FlodymArray(dims=dims[("t",)])

# 1B5
total_losses[...] = production * loss_rate

# 1B6
print(losses - total_losses)

## 1C: Stocks

- 1C1: Create a simple inflow-driven stock (dimension time) with scalar, log-normal lifetime with a 15 year mean and a 10 year standard deviation. 
- 1C2: Use `production` minus `losses` as inflow. Compute. Print the stock and outflow.


In [ ]:
# 1C1
lifetime = fd.lifetime_models.LogNormalLifetime(dims=dims["t",], mean=15, std=10)
dsm = fd.InflowDrivenDSM(dims=dims["t",], lifetime_model=lifetime)

# 1C2
dsm.inflow[...] = production - losses
dsm.compute()
print(dsm.stock)
print(dsm.outflow)

### Bonus: 
- 1C3: Do the same, but keep the element dimension. 

In [ ]:
# 1C3
lifetime_by_elem = fd.lifetime_models.LogNormalLifetime(dims=dims, mean=15, std=10)
dsm_by_elem = fd.InflowDrivenDSM(dims=dims, lifetime_model=lifetime_by_elem)
dsm_by_elem.inflow[...] = production - losses
dsm_by_elem.compute()
print(dsm_by_elem.stock)
print(dsm_by_elem.outflow)